In [ ]:
! cd ../../kamiwaza-sdk && pip install -e .

# Set up Kamiwaza Client SDK

In [ ]:
from kamiwaza_client import KamiwazaClient
from kamiwaza_client.schemas.ingestion import IngestionConfig
from kamiwaza_client.schemas.retrieval import RetrieveRelevantChunksRequest
from kamiwaza_client.services.retrieval import RetrievalService

import os

# Initialize the client
client = KamiwazaClient("http://localhost:7777/api/")

# Ingest our data. 
This will:
1. Load data into catalog
2. Chunk with offset tracking
3. Embed all chunks
4. Load into vectordb

In [ ]:
!ls './pet_store_docs'

In [ ]:
dataset_path = './pet_store_docs'
collection_name = "pet_store_docs_demo"

ingestion_config = IngestionConfig(
    dataset_path=dataset_path,
    description="pet_store_docs_demo",
    collection_name=collection_name,
    platform='file'
)

client.ingestion.ingest(ingestion_config)

# Do retrieval for a query

In [ ]:
query = "what is the vacation policy?"

retrieval_request = RetrieveRelevantChunksRequest(
    collections=[collection_name],
    query=query,
    max_results=5
)

In [ ]:
results = client.retrieval.retrieve_relevant_chunks(retrieval_request)

In [ ]:
for r in results:
    print(r)

# Do RAG ... that was easy

In [ ]:
deployments = client.serving.list_deployments()
valid_deployment = None
for deployment in deployments:
    if deployment.status == 'DEPLOYED' and deployment.instances:
        valid_deployment = deployment
        print(f"Found a deployment of {valid_deployment.m_name} at port {valid_deployment.lb_port} - using it")
        break
if valid_deployment is None:
    print("No valid deployments found.")

In [ ]:
import openai
from openai import OpenAI
import httpx

openai_client = OpenAI(
    api_key="local",
    base_url=f"http://localhost:{valid_deployment.lb_port}/v1",
    http_client=httpx.Client()
)

def rag_inference(query: str, collection_name: str, retrieval_service: RetrievalService):
    retrieval_request = RetrieveRelevantChunksRequest(
        collections=[collection_name],
        query=query,
        max_results=8
    )
    try:
        results = retrieval_service.retrieve_relevant_chunks(retrieval_request)
    except Exception as e:
        print(f"Failed to retrieve relevant chunks: {e}")
        return
    
    context_messages = [
        {"role": "system", "content": "You are a helpful assistant. Here are relevant pieces of information. Be concise."}
    ]
    referenced_files = set()
    
    for chunk in results.chunks:
        context_messages.append(
            {"role": "system", "content": f"Source: {chunk.source}, Chunk ID: {chunk.offset}, Data: {chunk.data}"}
        )
        referenced_files.add(chunk.source)
    
    context_messages.append({"role": "user", "content": query})
    
    try:
        chat_completion = openai_client.chat.completions.create(
            model="local-model",
            messages=context_messages,
            max_tokens=150
        )
        response_content = chat_completion.choices[0].message.content
    except Exception as e:
        print(f"An error occurred during inference: {e}")
        return
    
    print("AI Response:")
    print(response_content)
    print("\nReferenced Files:")
    for file in referenced_files:
        print(f"- {file}")

In [ ]:
def default_inference(query: str):
    """
    Make a direct inference call to the LLM without RAG context.
    
    Args:
        query (str): The user's query to send to the model
    """
    try:
        chat_completion = openai_client.chat.completions.create(
            model="local-model",
            messages=[
                {"role": "system", "content": "You are a helpful assistant. Be concise. Do not hallucinate. If you do not know the answer, tell the user."},
                {"role": "user", "content": query}
            ],
            max_tokens=150
        )
        response_content = chat_completion.choices[0].message.content
    except Exception as e:
        print(f"An error occurred during inference: {e}")
        return
    
    print("AI Response:")
    print(response_content)

In [ ]:
default_inference("what is the vacation policy?")


In [ ]:
rag_inference("what is the vacation policy?", collection_name, client.retrieval)
